In [1]:
import re
import urllib.request
from bs4 import BeautifulSoup
import time
import pymysql
import torch
from torchtext import data
#import re
import torchtext
import os
import bs4

In [2]:
from konlpy.tag import Okt
okt = Okt()

C:\Users\samsung\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [3]:
twitter = Okt()
def my_tokenizer(text):
    return_value = list(map(lambda x:x[0],okt.pos(text))) # okt.pos(text) output: [(word,kinds)], ex[("진짜","Noun")] I want only word(each tuple index 0)
    if len(return_value)==0:#if length zero error ouccur
        return_value.append(".") 
    return return_value

def morphs_no_zero(text):
    token = Okt().morphs(text)
    if len(token) == 0:
        token.append(".")
    return token
# def my_token(text): #test user func
#   return text.split()
dir_path = "./trained_model"
SEED = 1234
#TEXT = torch.load(os.path.join(dir_path,"TEXT.pt"))
TEXT = torch.load(dir_path+"/TEXT.pt")
TEXT.vocab = torch.load(dir_path+"/TEXT_vocab.pt")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text,  text_lengths):

        embedded = self.dropout(self.embedding(text))
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        return self.fc(hidden)
    
def predict_sentiment(model, sentence):
    model.eval()
    tokenized = morphs_no_zero(sentence)
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()
    
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 200
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

import os
model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

# path_dir = "/content/gdrive/My Drive/Colab Notebooks/datamining/Models/"
# pretrained_model_list = os.listdir(path_dir)
# pretrained_model_list.sort()   #가장 loss가 낮은 모델을 부르기 위해 정렬 #저장양식 model_name + "-loss_"+str(int(loss*1000)) ex LSTM_sequential_loss_1 #모델/loss 구분 "-"(하이푼)
# i = 0
# #model_state = model.state_dict()
# pre_train_loss = 1
# for trained_model in pretrained_model_list:
#   i+=1
#   if trained_model.split("-")[0] == "sentimental_analysis_model":
#     #model_state = torch.load(path_dir+"/"+trained_model)["model_state"]
#     model.load_state_dict(torch.load(path_dir+"/"+trained_model)["model_state"])
#     pre_train_loss = torch.load(path_dir+"/"+trained_model)["loss"]
#     print("불러온 모델",trained_model, "pre_train_loss: ",pre_train_loss)   
#     break
#   if i == len(pretrained_model_list):
#     print("저장된 같은 모델이 없습니다")
model.load_state_dict(torch.load(dir_path +"/sentimental_analysis_model-loss_474.pt",map_location=torch.device('cpu'))["model_state"])
model = model.to(device)

In [4]:
class Review:
    def __init__(self, star, date, title, visit, recommend, text, movie_title):
        self.star = star
        self.date = date
        #self.year, self.month, self.day = self.date.spilt('.')
        self.title = title
        self.visit = visit
        self.recommend = recommend
        self.text = text.replace("'","")
        self.movie_title = movie_title.replace("'","")
    def __str__(self):
        review_info = "별점: " + str(self.star) + "\n" +\
                    "날짜: " + self.date + "\n" + \
                    "제목: " + self.title + "\n" + \
                    "조회: " + str(self.visit) + "\n" + \
                    "추천: " + str(self.recommend) + "\n" +\
                    "내용: " + self.text + "\n"
        return review_info

In [ ]:
def movie_in_url(url):
    """검색된 페이지에서 네이버 영화로 들어가는 url을 return"""
    soup = BeautifulSoup(urllib.request.urlopen(url).read(), "html.parser")
    return soup.find("div", class_="info_main").find("h3").find("a").get("href")
def review_in_url(url):
    """네이버 영화에서 review로 가는 url과 page수, url에 들어가는 영화code를 리턴"""
    soup = BeautifulSoup(urllib.request.urlopen(url).read(), "html.parser")
    div = soup.find("div", class_ = "sub_tab_area")
    data_list = div.select("ul li")
    
    url_code = re.findall("\d+", url)[0] #url_code 추출
    
    
    url_path = ""
    root_url = "https://movie.naver.com/movie/bi/mi"
    for data in data_list:
        #print(data.find("a").get("href"))
        url_path = data.find("a").get("href")
        if "review" in url_path:
            url_path= root_url+ url_path[1:]  #"."을 루트경로로 바꿔줌
            break
    soup = BeautifulSoup(urllib.request.urlopen(url_path).read(), "html.parser")
    review_count = int(soup.find("div",class_="top_behavior").find("em").text.replace(",",""))
    page_count = review_count // 10 + 1
    return url_path, page_count, url_code
def rank_serch_url(url):
    """네이버에 검색해서 영화 순위의 각 검색 url을 list로 리턴"""
    soup = BeautifulSoup(urllib.request.urlopen(url).read(), "html.parser")
    div = soup.find("div", class_="_content")
    data_list = div.select("ul li")
    naver_search_list = []
    pre_url = "https://search.naver.com/search.naver"
    
    for data in data_list:
        #print(data.find("div", class_ = "movie_info").find("a").get("href"))
        af_url = data.find("div", class_ = "movie_info").find("a").get("href")
        naver_search_list.append(pre_url+ af_url)
    #print(naver_search_list)
    return naver_search_list
def review_num(url):
    """review 본문으로 접근하는 url number를 리스트로 반환"""
    soup = BeautifulSoup(urllib.request.urlopen(url).read(), "html.parser")
    div = soup.find("div", class_="review")
    data_list = div.select("ul li")
    numbers = []
    for data in data_list:
        #print(data.find('a').get('onclick'))
        onclick = data.find('a').get('onclick')
        #print(onclick)
        if re.findall("\d+", onclick) != []:         #url에 들어가는 넘버만 획득
            number = re.findall("\d+", onclick)[0]
            numbers.append(number)
    return numbers
def one_page_crawling(url_code, numbers):
    global cur
    review_list = []
    review_root_url ="https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid="
    end_url = "&code="+str(url_code)+"&order=#tab" 
    for number in numbers:
       # try:
            review_url = review_root_url + str(number) + end_url
            print(review_url)
            review_soup = BeautifulSoup(urllib.request.urlopen(review_url).read(), "html.parser")
            movie_title = review_soup.find("div", "mv_info").find("h3").find("a").text
            #print(movie_title)
            div = review_soup.find("div", class_ = "review")
            title = div.find("strong", class_="h_lst_tx").text.strip()
            date = div.find("span", class_ = "wrt_date").text.strip()
            text_area = div.find("div", class_ = "user_tx_area")
            #print(text.select("span"))
            try:
                star = div.find("div","star_score").find("em").text.strip()
            except:
                star = 0
            #print(star)
            visit = div.find("div",class_ = "user_tx_info").find("em").text.strip()
            #print(visit)
            recommend = div.find("div",class_ = "user_tx_info").find("em",id ="goodReviewCount").text.strip()
            #print(recommend)
            #print(text_area.text)
            text = text_area.text.strip()
            #print("text:",text)
            #print(Review(star, date, title, visit, recommend, text))
            review_list.append(Review(star, date, title, visit, recommend, text,movie_title))
        #except: continue
            
    return review_list
def search_url(title):
    baseurl = 'https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query='
    url = baseurl + urllib.parse.quote_plus(title)
    return url
def CRAWLING(txt):
        root_url = "https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=%EC%98%81%ED%99%94%EC%88%9C%EC%9C%84"
        #url_list = rank_serch_url(root_url)  ###현재 영화 순위로 검색시

        url_list = []
        movie_list = []
        #여기에서 txt받아와서 movie_list에 append
        #movie_list = ["겨울왕국","나를 찾아줘"]  ### 그냥 검색타이틀로 크롤링 할시
        movie_list.append(txt)

        for movie in movie_list:
            url_list.append(search_url(movie))

        url_code = 0
        page_count = 1
        all_review = []
        for rank_url in url_list:
            url = movie_in_url(rank_url)
            url, page_count, url_code = review_in_url(url)

            maximum_page = 1
            for page in range(1,page_count+1):
                if page > maximum_page:
                    break
                url2 = url + "&page="+str(page)
                #print(url2)
                #print(url, page_count, url_code)
                #print(review_num(url))
                numbers = review_num(url2)
                review_list = one_page_crawling(url_code, numbers)
                all_review.extend(review_list)
        return all_review

In [ ]:
import sys
from PyQt5.QtWidgets import *
from PyQt5 import uic
from PyQt5.QtGui import *
from PyQt5.QtCore import *

form_class = uic.loadUiType("movie.ui")[0]

class MyWindow(QMainWindow, form_class):
    def __init__(self):
        super().__init__()
        self.setupUi(self)
        self.pushButton.clicked.connect(self.btn_clicked)
        self.pushButton_2.clicked.connect(self.Next)#next
        self.pushButton_3.clicked.connect(self.Prev)#prev

    def btn_clicked(self):
        txt = self.lineEdit.text()
        url = CRAWLING(txt)
        
        all_reviews = []
        title_context = []
        self.title_and_context= []
        for review in url:
            all_reviews.append(str(review).split('\n'))
        for i in all_reviews:
            temp2=[]
            for j in i: 
                if '제목' in j:
                    temp2.append(j)
                elif '내용' in j:
                    temp2.append(j)
            title_context.append(temp2)
        for i in title_context:
            temp3 = []
            for j in i: 
                if '내용' in j:
                    temp3.append(j.split('.'))
                else:
                    temp3.append(j)
            self.title_and_context.append(temp3)
        self.index = 0
        self.textBrowser.clear()
        #첫번째 데이터 
        self.textBrowser.setTextColor(QColor(Qt.black))
        self.textBrowser.append(self.title_and_context[0][0])
        for l2 in self.title_and_context[0][1]:
            pred = predict_sentiment(model, l2)
            if pred >= 0.95:
                #if 긍정 : settextcolor blue
                self.textBrowser.setTextColor(QColor(Qt.blue))
                self.textBrowser.append(l2)
            elif pred <= 0.05:
                #else 부정 : settextcolor red
                self.textBrowser.setTextColor(QColor(Qt.red))
                self.textBrowser.append(l2)
            else:
                self.textBrowser.setTextColor(QColor(Qt.black))
                self.textBrowser.append(l2)
                
        QMessageBox.about(self, "message", "크롤링 완료")#lineEdit에 쓰는 문자열 가져오기
       
                
   
        
    def Next(self):
        #QMessageBox.about(self, "message", "다음")
        self.textBrowser.clear()
        self.index += 1;
        if(self.index >9):
            self.index = 0
        self.textBrowser.setTextColor(QColor(Qt.black))
        self.textBrowser.append(self.title_and_context[self.index][0])
        for l2 in self.title_and_context[self.index][1]:
            pred = predict_sentiment(model, l2)
            if pred >= 0.95:
                #if 긍정 : settextcolor blue
                self.textBrowser.setTextColor(QColor(Qt.blue))
                self.textBrowser.append(l2)
            elif pred <= 0.05:
                #else 부정 : settextcolor red
                self.textBrowser.setTextColor(QColor(Qt.red))
                self.textBrowser.append(l2)
            else:
                self.textBrowser.setTextColor(QColor(Qt.black))
                self.textBrowser.append(l2)
        
    def Prev(self):
        #QMessageBox.about(self, "message", "이전")
        self.textBrowser.clear()
        self.index -= 1;
        if(self.index <0):
            self.index = 9
        self.textBrowser.setTextColor(QColor(Qt.black))
        self.textBrowser.append(self.title_and_context[self.index][0])
        for l2 in self.title_and_context[self.index][1]:
            pred = predict_sentiment(model, l2)
            if pred >= 0.95:
                #if 긍정 : settextcolor blue
                self.textBrowser.setTextColor(QColor(Qt.blue))
                self.textBrowser.append(l2)
            elif pred <= 0.05:
                #else 부정 : settextcolor red
                self.textBrowser.setTextColor(QColor(Qt.red))
                self.textBrowser.append(l2)
            else:
                self.textBrowser.setTextColor(QColor(Qt.black))
                self.textBrowser.append(l2)
    
if __name__ == "__main__":
    app = QApplication(sys.argv)
    myWindow = MyWindow()
    myWindow.show()
    app.exec_()
   

In [11]:
if __name__ == "__main__":
    root_url = "https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=%EC%98%81%ED%99%94%EC%88%9C%EC%9C%84"
    #url_list = rank_serch_url(root_url)  ###현재 영화 순위로 검색시
#################################################
    url_list = []
    movie_list = []
    #여기에서 txt받아와서 movie_list에 append
    #movie_list = ["겨울왕국","나를 찾아줘"]  ### 그냥 검색타이틀로 크롤링 할시
    movie_list.append('겨울왕국')
    
    for movie in movie_list:
        url_list.append(search_url(movie))
 ###########################################   
    url_code = 0
    page_count = 1
    all_review = []
    for rank_url in url_list:
        url = movie_in_url(rank_url)
        url, page_count, url_code = review_in_url(url)

        maximum_page = 1
        for page in range(1,page_count+1):
            if page > maximum_page:
                break
            url2 = url + "&page="+str(page)
            print(url2)
            #print(url, page_count, url_code)
            #print(review_num(url))
            numbers = review_num(url2)
            review_list = one_page_crawling(url_code, numbers)
            all_review.extend(review_list)

https://movie.naver.com/movie/bi/mi/review.nhn?code=136873&page=1
https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid=4728545&code=136873&order=#tab
https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid=4728583&code=136873&order=#tab
https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid=4728663&code=136873&order=#tab
https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid=3833889&code=136873&order=#tab
https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid=4728656&code=136873&order=#tab
https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid=4728590&code=136873&order=#tab
https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid=4728991&code=136873&order=#tab
https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid=4729315&code=136873&order=#tab
https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid=4728506&code=136873&order=#tab
https://movie.naver.com/movie/bi/mi/reviewread.nhn?nid=4728652&code=136873&order=#tab


In [6]:
txt = "엘사 옷은 예술이죠"
pred = predict_sentiment(model, txt)
if round(pred):
    print("긍정")
else:
    print("부정")
print(pred)

부정
0.01667693629860878


In [23]:
len(all_review)

10

In [36]:
print(str(all_review[0]))

별점: 0
날짜: 2019.11.22
제목: 겨울왕국2 해석 (Frozen2, 2019)
조회: 3260
추천: 15
내용: 스토리 핵심은 00입니다.※영화를 보지 않은 분들에게는 스포일러가 될 수 있으니 주의 부탁드립니다내용의 전반적으로 사랑으로 주제가 연결됩니다.영화 시작에 어린 안나와 엘사가 눈으로 만드는 내용 보셨나요?사랑시작하자 마자사랑으로 행복한 이야기를 아이들은 만듭니다.첫번째 사랑은 남녀간의 사랑입니다.그리고 엘사 마법은적이였던 엘사의 어머니가 아버지를 구했기 때문에생겼습니다.아아아아~ 하는 소리는엘사 어머니가 아버지를 살리기 위해도움을 요청할 때의 소리입니다.그 사랑의 소리가엘사를 안내하죠.두번째 사랑은 Number2 에서 강조합니다."Some Things Never Change"올라프는 나중에 깨닫죠.변하지 않는 것은 사랑이라는 사실을노래에서 전반적으로 강조합니다.함께해야 한다는 사실두번째 사랑은 우정입니다.사랑의 주제 3번째는 나 자신의 사랑입니다.타이틀 곡인 Show yourself 에서 확연히 드러납니다.너를 보여줘.내가 찾아낸 정답이 바로 너야.자신를 찾으면서 더 행복해지는 엘사를 볼 수 있습니다."Step into the power"힘으로 다가가는 모습으로이전까지 두려워했던 자신의 힘(얼음)을받아드리는 모습을 보입니다.엘사는 노래 부르죠."You are the one youve been waiting for네가 기다리는 건 바로 네 자신이야.자신을 사랑하게 되는 장면입니다.그리고 옷도 바뀌죠.3번째 주제는 나 자신의 사랑입니다.4번째 주제는 가족의 사랑입니다.엘사의 부모님은 엘사의 비밀을 알기위해목숨을 걸고 항해를 하죠.부모님과 자녀의 사랑입니다.이처럼 사랑이라는 주제로다양하게 이야기합니다.(이외에도 자연과 인간, 자매간의 우정, 진실 등도 있지만)사랑의 전형인 결혼으로스토리는 마무리 됩니다.결론은 사랑입니다.겨울왕국2를 보면 디즈니는 잘하는 것을 잘해냅니다.엘사 옷은 예술이죠.  영상미가 훌륭합니다.특히 올라프는 톡톡한 역할을 해내

데이터 정제

In [12]:
all_reviews = []
title_context = []
for review in all_review:
    all_reviews.append(str(review).split('\n'))
for i in all_reviews:
    temp2=[]
    for j in i: 
        if '제목' in j:
            temp2.append(j)
        elif '내용' in j:
            temp2.append(j)
    title_context.append(temp2)
title_context

[['제목: 겨울왕국2 해석 (Frozen2, 2019)',
  '내용: 스토리 핵심은 00입니다.※영화를 보지 않은 분들에게는 스포일러가 될 수 있으니 주의 부탁드립니다내용의 전반적으로 사랑으로 주제가 연결됩니다.영화 시작에 어린 안나와 엘사가\xa0눈으로 만드는 내용 보셨나요?사랑시작하자 마자사랑으로 행복한 이야기를 아이들은 만듭니다.첫번째 사랑은 남녀간의 사랑입니다.그리고 엘사 마법은적이였던 엘사의 어머니가 아버지를 구했기 때문에생겼습니다.아아아아~ 하는 소리는엘사\xa0어머니가 아버지를 살리기 위해도움을 요청할\xa0때의 소리입니다.그 사랑의 소리가엘사를 안내하죠.두번째 사랑은\xa0Number2 에서 강조합니다."Some Things Never Change"올라프는 나중에 깨닫죠.변하지 않는 것은 사랑이라는 사실을노래에서 전반적으로 강조합니다.함께해야 한다는 사실두번째 사랑은 우정입니다.사랑의 주제 3번째는 나 자신의 사랑입니다.타이틀 곡인 Show yourself 에서 확연히 드러납니다.너를 보여줘.내가 찾아낸 정답이 바로 너야.자신를 찾으면서 더 행복해지는 엘사를 볼 수 있습니다."Step into the power"힘으로 다가가는 모습으로이전까지 두려워했던 자신의 힘(얼음)을받아드리는 모습을 보입니다.엘사는 노래 부르죠."You are the one youve been waiting for네가 기다리는 건 바로 네 자신이야.자신을 사랑하게 되는 장면입니다.그리고 옷도 바뀌죠.3번째 주제는 나 자신의 사랑입니다.4번째 주제는 가족의 사랑입니다.엘사의 부모님은 엘사의 비밀을 알기위해목숨을 걸고 항해를 하죠.부모님과 자녀의 사랑입니다.이처럼 사랑이라는 주제로다양하게 이야기합니다.(이외에도 자연과 인간, 자매간의 우정, 진실\xa0등도 있지만)사랑의 전형인 결혼으로스토리는 마무리 됩니다.결론은 사랑입니다.겨울왕국2를 보면 디즈니는 잘하는 것을 잘해냅니다.엘사 옷은 예술이죠.\xa0 영상미가 훌륭합니다.특히 올라프는 톡톡한 역할을 해내죠.웃음의 핵심 역할

In [17]:
title_and_context = []
for i in title_context:
    temp3 = []
    for j in i: 
        if '내용' in j:
            temp3.append(j.split('.'))
        else:
            temp3.append(j)
    title_and_context.append(temp3)
for l1 in title_and_context:
    print(l1[0])
    for l2 in l1[1]:
        print(l2)       

제목: 겨울왕국2 해석 (Frozen2, 2019)
내용: 스토리 핵심은 00입니다
※영화를 보지 않은 분들에게는 스포일러가 될 수 있으니 주의 부탁드립니다내용의 전반적으로 사랑으로 주제가 연결됩니다
영화 시작에 어린 안나와 엘사가 눈으로 만드는 내용 보셨나요?사랑시작하자 마자사랑으로 행복한 이야기를 아이들은 만듭니다
첫번째 사랑은 남녀간의 사랑입니다
그리고 엘사 마법은적이였던 엘사의 어머니가 아버지를 구했기 때문에생겼습니다
아아아아~ 하는 소리는엘사 어머니가 아버지를 살리기 위해도움을 요청할 때의 소리입니다
그 사랑의 소리가엘사를 안내하죠
두번째 사랑은 Number2 에서 강조합니다
"Some Things Never Change"올라프는 나중에 깨닫죠
변하지 않는 것은 사랑이라는 사실을노래에서 전반적으로 강조합니다
함께해야 한다는 사실두번째 사랑은 우정입니다
사랑의 주제 3번째는 나 자신의 사랑입니다
타이틀 곡인 Show yourself 에서 확연히 드러납니다
너를 보여줘
내가 찾아낸 정답이 바로 너야
자신를 찾으면서 더 행복해지는 엘사를 볼 수 있습니다
"Step into the power"힘으로 다가가는 모습으로이전까지 두려워했던 자신의 힘(얼음)을받아드리는 모습을 보입니다
엘사는 노래 부르죠
"You are the one youve been waiting for네가 기다리는 건 바로 네 자신이야
자신을 사랑하게 되는 장면입니다
그리고 옷도 바뀌죠
3번째 주제는 나 자신의 사랑입니다
4번째 주제는 가족의 사랑입니다
엘사의 부모님은 엘사의 비밀을 알기위해목숨을 걸고 항해를 하죠
부모님과 자녀의 사랑입니다
이처럼 사랑이라는 주제로다양하게 이야기합니다
(이외에도 자연과 인간, 자매간의 우정, 진실 등도 있지만)사랑의 전형인 결혼으로스토리는 마무리 됩니다
결론은 사랑입니다
겨울왕국2를 보면 디즈니는 잘하는 것을 잘해냅니다
엘사 옷은 예술이죠
  영상미가 훌륭합니다
특히 올라프는 톡톡한 역할을 해내죠
웃음의 핵심 역할입니다
그 올라프의 역할은 이해입니다
이해올라프

In [ ]:
title_and_context

In [24]:
connect = pymysql.connect(host='localhost', user= 'root', password='8547', db='review', charset='utf8')
cursor = connect.cursor()

In [ ]:
for review in all_review:
    